Ein Zufallszahlengenerator soll 300 reelle gleichverteilte Zufallszahlen im Intervall $[1, 7]$ liefern. Die
berechneten Pseudozufallszahlen verteilten sich gemäß
- Klasse $t_i \in ...$ [1, 2)&emsp;[2, 3)&emsp;[3, 4)&emsp;[4, 5)&emsp;[5, 6)&emsp;[6, 7)
- Häufigkeit &emsp;&emsp;42 &emsp;&emsp;56 &emsp;&emsp;71 &emsp;&emsp;45&emsp;&emsp;39&emsp;&emsp;47

Teste unter einer Irrtumswahrscheinlichkeit von $α = 0.05$, ob es sich um gleichverteilte Zufallszahlen
handelt.

In [40]:
from sympy.stats import P, DiscreteUniform
from scipy import stats
import plotly.graph_objects as go
from numpy import transpose

classes = [[1,2], [2,3], [3,4], [4,5], [5,6], [6,7]]
delta_t_amounts = [42,56,71,45,39,47]
n = 300
alpha = 0.05

class sample:
    n = n

    def __init__(self, lowerLimit_closed, upperLimit_open, nᵢ):
        self.lowerLimit = lowerLimit_closed
        self.upperLimit = upperLimit_open
        self.nᵢ = nᵢ
        self.X = DiscreteUniform('X', list(range(max(min(classes)), max(max(classes))+ 1))) # set lowest and highest element as boundary
        self.pᵢ = float(P(self.X <= self.upperLimit) - P(self.X <= self.lowerLimit))
        self.nᵢˣ = sample.n * self.pᵢ
        self.equation = (self.nᵢ - self.nᵢˣ)**2/self.nᵢˣ

def sum_col(name, table_data):
    return sum([table_data[i][name] for i, item in enumerate(table_data)])

if n != sum(delta_t_amounts):
    raise IOError("delta_t_amounts and n are not equal, possibly wrong input")

s = [sample(delta_t_class[0], delta_t_class[1], delta_t_amounts[i]) for i, delta_t_class in enumerate(classes)]

table_data = []
for i, _sample in enumerate(s):
    samples = {}
    samples['class'] = f"[{_sample.lowerLimit},{_sample.upperLimit})"
    samples['nᵢ'] = _sample.nᵢ
    samples['pᵢ'] = _sample.pᵢ
    samples['nᵢˣ'] = _sample.nᵢˣ
    samples['equation'] = _sample.equation
    table_data.append(samples)
table_data.append({'class': 'sum', 'nᵢ': sum_col('nᵢ', table_data), 'pᵢ': sum_col('pᵢ', table_data), 'nᵢˣ': sum_col('nᵢˣ', table_data), 'equation': sum_col('equation', table_data)})

f = len(delta_t_amounts) -1
quantile = 1-alpha
degrees = f #-1 # 1 assumption -> degrees -1
c = stats.chi2.ppf(quantile, degrees)
z = table_data[-1]['equation']

print(f"test value: z = {z}")
print(f'c = {c} is the {quantile:1f}-quantile of the 𝜒²-distribution with f = {degrees} degrees')
print(f"the sample {'seems' if z < c else 'does not seem'} to be equally distributed with a {alpha*100}% error rate")

fig = go.Figure(data=[go.Table(header=dict(values=list(samples.keys())),
                 cells=dict(values=transpose([list(item.values()) for item in table_data]), align="left"))
                    ])
fig.show()

test value: z = 13.92
c = 11.070497693516351 is the 0.950000-quantile of the 𝜒²-distribution with f = 5 degrees
the sample does not seem to be equally distributed with a 5.0% error rate


# Aufgabe 2
In Beispiel 2.60 der Vorlesung hatten wir gezeigt, dass die 100 Pseudozufallszahlen   
49, 50, 51, 51, 56, 57, 57, 58, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 62, 62, 62, 62, 62, 62, 62, 63, 63, 64, 64, 64, 65, 65, 65, 66, 66, 66, 66, 66, 67, 67, 67, 68, 68, 68, 68, 68, 68, 69, 69, 69, 69, 69, 69, 69, 70, 70, 70, 70, 71, 71, 71, 71, 71, 71, 72, 73, 73, 73, 73, 73, 73, 73, 74, 74, 74, 74, 74, 74, 74, 74, 74, 75, 76, 76, 76, 76, 76, 76, 77, 77, 78, 78, 80, 81, 82, 82, 83, 84, 84, 86   
bei einer Irrtumswahrscheinlichkeit von 5% nicht normalverteilt zu den Parametern $µ = 70$ und $σ = 5$
sind. Teste nun unter derselben Irrtumswahrscheinlichkeit, ob sie überhaupt normalverteilt sind.

In [61]:
from sympy import oo, sqrt
from sympy.stats import P, Normal
from scipy import stats
import plotly.graph_objects as go

data = [49, 50, 51, 51, 56, 57, 57, 58, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 62, 62, 62, 62, 62, 62, 62, 63, 63, 64, 64, 64,
65, 65, 65, 66, 66, 66, 66, 66, 67, 67, 67, 68, 68, 68, 68, 68, 68, 69, 69, 69, 69, 69, 69, 69, 70, 70, 70, 70,
71, 71, 71, 71, 71, 71, 72, 73, 73, 73, 73, 73, 73, 73, 74, 74, 74, 74, 74, 74, 74, 74, 74, 75, 76, 76, 76, 76, 76, 76,
77, 77, 78, 78, 80, 81, 82, 82, 83, 84, 84, 86]
classes = [[-oo,57], [57,62], [62,67], [67,72], [72,77], [77,82], [82,oo]]
delta_t_amounts = [5,13,20,26,24,6,6]
n = 100
assumptions = 2
alpha = 0.05

mu = sum(data)/len(data)
s_squared = 1/(n-1)*sum(list(map(lambda x: (x - mu)**2, data)))


class sample:
    n = n

    def __init__(self, lowerLimit_closed, upperLimit_open, nᵢ):
        self.lowerLimit = lowerLimit_closed
        self.upperLimit = upperLimit_open
        self.nᵢ = nᵢ
        self.X = Normal('X', mu, sqrt(s_squared)) # set lowest and highest element as boundary
        self.pᵢ = float(P(self.X <= self.upperLimit) - P(self.X <= self.lowerLimit))
        self.nᵢˣ = sample.n * self.pᵢ
        self.equation = (self.nᵢ - self.nᵢˣ)**2/self.nᵢˣ

def sum_col(name, table_data):
    return sum([table_data[i][name] for i, item in enumerate(table_data)])

if n != sum(delta_t_amounts):
    raise IOError("delta_t_amounts and n are not equal, possibly wrong input")

s = [sample(delta_t_class[0], delta_t_class[1], delta_t_amounts[i]) for i, delta_t_class in enumerate(classes)]

table_data = []
for i, _sample in enumerate(s):
    samples = {}
    samples['class'] = f"[{_sample.lowerLimit},{_sample.upperLimit})"
    samples['nᵢ'] = _sample.nᵢ
    samples['pᵢ'] = _sample.pᵢ
    samples['nᵢˣ'] = _sample.nᵢˣ
    samples['equation'] = _sample.equation
    table_data.append(samples)
table_data.append({'class': 'sum', 'nᵢ': sum_col('nᵢ', table_data), 'pᵢ': sum_col('pᵢ', table_data), 'nᵢˣ': sum_col('nᵢˣ', table_data), 'equation': sum_col('equation', table_data)})

f = len(delta_t_amounts) -1
quantile = 1-alpha
degrees = f - assumptions
c = stats.chi2.ppf(quantile, degrees)
z = table_data[-1]['equation']

print(f"test value: z = {z}")
print(f'c = {c} is the {quantile:1f}-quantile of the 𝜒²-distribution with f = {degrees} degrees')
print(f"the sample {'seems' if z < c else 'does not seem'} to be equally distributed with a {alpha*100}% error rate")

fig = go.Figure(data=[go.Table(header=dict(values=list(samples.keys())),
                 cells=dict(values=transpose([list(item.values()) for item in table_data]), align="left"))
                    ])
fig.show()

test value: z = 4.218097871347002
c = 9.487729036781154 is the 0.950000-quantile of the 𝜒²-distribution with f = 4 degrees
the sample seems to be equally distributed with a 5.0% error rate
